In [1]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from tqdm import tqdm_notebook, tnrange
import pickle
from termcolor import colored
import sqlite3

In [2]:
conn = sqlite3.connect("Dane/wikipedyjka2.db")

In [4]:
def to_lemma_mapping():
    all_lemmas = dd(list)
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[:2]
        all_lemmas[L[1].lower()].append(L[0].lower())
    return all_lemmas
lemma_mapping = to_lemma_mapping()

In [5]:
wiki_list = []
key = ""
with open("Dane/fp_wiki.txt") as f:
    cntr = -1
    for line in f:
        tokenized_line = line.split(": ")
        if tokenized_line[0] == "TITLE":
            if tokenized_line[-1][-1] == "\n":
                tokenized_line[-1] = tokenized_line[-1][:-1]
            cntr+=1
            wiki_list.append((tokenized_line[-1], []))
            continue
        wiki_list[-1][1].append(line)

In [6]:
cntr1 = 0
title_positional_index = dd(list)
article_positional_index = dd(list)

for title, article in wiki_list:
    tokenized_line = nltk.word_tokenize(title)
    cntr2 = 0
    for w in tokenized_line:
        for lemma in lemma_mapping[w]:
            title_positional_index[lemma].append((cntr1, cntr2))
        cntr2 +=1
    cntr2 = 0
    for line in article:
        tokenized_line = nltk.word_tokenize(line)
        for w in tokenized_line:
            for lemma in lemma_mapping[w]:
                article_positional_index[lemma].append((cntr1, cntr2))
            cntr2 += 1
    cntr1 += 1

KeyboardInterrupt: 

In [ ]:
c = conn.cursor()
for k, v in title_positional_index:
    c.execute(f'''CREATE TABLE {k}_title
             (title_position INTEGER, word_position INTEGER)''')
    for ind in v:
        c.execute(f"INSERT INTO {k}_title VALUES ({ind[0]}, {ind[1]})")
c.commit()
for k, v in article_positional_index:
    c.execute(f'''CREATE TABLE {k}_title
             (title_position INTEGER, word_position INTEGER)''')
    for ind in v:
        c.execute(f"INSERT INTO {k}_title VALUES ({ind[0]}, {ind[1]})")
c.commit()